In [15]:
import pickle
import os
from glob import glob
import pandas as pd

In [11]:
len(os.listdir('data/scraped'))

14

In [63]:
filepaths = glob('data/scraped/*.pickle')

In [64]:
jobs = []
for filepath in filepaths:
    filename = filepath.split('\\')[-1]
    with open(filepath, 'rb') as file:
        loaded = pickle.load(file)
        for job in loaded:
            if 'Type' not in job.keys():
                job['Type'] = None
        with open(filepath, 'wb') as file:
            pickle.dump(loaded, file)

In [43]:
n = 0
for job in jobs:
    if 'Type' not in job.keys():
        print(job['Title'])
        n += 1
print(n)

Graphic Design Intern
Grapghic Designer PNG
2023 bp Geophysics Internship Programme (For 2024 Graduate)
Senior Local Designer
3D Modeler
Graphic Design Intern
Key Account Buyer
Affiliate Manager
FIBI APAC CI/VC FSQR Leader
Technical Account Manager, Strategic Partnerships (Integration/API)(Bangkok-based, Relocation Provided)
Enterprise Account Executive
Account Executive - Surabaya
FIBI APAC CI/VC FSQR Leader
Country PR Lead - Indonesia
FIBI APAC CI/VC FSQR Leader
Sales Executive - Bojonegoro
Territory Account Sales Executive, Indonesia
FIBI APAC CI/VC FSQR Leader
SME Account Executive
Technical Sales Executive - Food Ingredients
Account Executive - Indonesia
FIBI APAC CI/VC FSQR Leader
FIBI APAC CI/VC FSQR Leader
Senior Accountant
Sales Lead Indonesia
Account Management, Fashion (e-commerce)
Business Development Manager (PS/Gov/FSI focus)
REL Lead
Head of Sales
Accounting Manager (Bangkok based, relocation provided)
Pricing Manager
Technical Service Executive - Industrial Services
FIB

In [32]:
for i, job in enumerate(jobs):
    try:
        print(i, job['Title'], '>', job['Type'])
    except KeyError:
        job['Type'] = None
        print(i, job['Title'], '> "add None job type."')

0 Senior 3D Civil Designer > Di Kantor
1 Senior Architect > Di Kantor
2 Coord-Graphic Design > Di Kantor
3 3D Designer – Piping and Mechanical (Plant 3D Admin) > Gabungan
4 Technical Assistant (Architectural Drafter) > Di Kantor
5 3D Designer, CLO > Di Kantor
6 Level Designer > Gabungan
7 Apprenticeship - Internal Communication Designer - Pasuruan > Di Kantor
8 Business & Integration Architecture Analyst > Di Kantor
9 Planner > Di Kantor
10 3D Modeler > Di Kantor
11 Product Designer FTW, Originals > Di Kantor
12 Graphic Designer > Di Kantor
13 Personal Designer > Di Kantor
14 ID - Head of Label & Artist Solutions (Internal Consultant) > Di Kantor
15 Graphic Design Intern > Gabungan
16 Designer > Di Kantor
17 Creative Design Intern > Di Kantor
18 Digital Consultant (UI / UX) (Indonesia) > Di Kantor
19 Senior UX Researcher > Gabungan
20 Graphic Design Intern > "add None job type."
21 Apprenticeship - Content and Visual Designer > Di Kantor
22 3D MODELLER > Di Kantor
23 Senior UX Research

In [38]:
df_job = pd.DataFrame({
    'Title': [job['Title'] for job in jobs],
    'Company': [job['Company'] for job in jobs],
    'Area': [job['Area'] for job in jobs],
    'Type': [job['Type'] for job in jobs],
    'Skills': [job['Skills'] for job in jobs],
    'Description': [job['Description'] for job in jobs],
})

In [40]:
df_job.isna().sum()

Title             0
Company           0
Area              0
Type            462
Skills         1630
Description       0
dtype: int64